# Synthesis Analsysis
In this notebook, we analyze the synthesis results of the [parameter sweeping](https://sdrangan.github.io/fpgademos/loopopt/paramsweep.html).  As discussed there, we synthesize multiple versions of the vector multiplier with different unroll factors.  Here, we will parse the synthesis results to look at the resource usage for different unrolling factors.

 

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import os

## Loading the synthesis output files

Follow the steps in the [parameter sweeping](https://sdrangan.github.io/fpgademos/loopopt/paramsweep.html) instructions.  This will run the synthesis and create a number of synthesis xml files stored in the directory `report_dir` below.  If you have run the synthesis on a remote machine, you will need to manually copy these files to wherever you are running this notebook.  We first go through the directory to find he report files.

In [ ]:
report_dir = os.path.join('..', 'vmult_reports')

'..\\vmult_reports'

In [24]:
import re

pattern = re.compile(r"^mult_loop_csynth_uf(\d+)\.xml")
mult_loop_files = []

for fname in os.listdir(report_dir):
    m = pattern.match(fname)
    if m:
        mult_loop_files.append((fname, int(m.group(1))))

mult_loop_files.sort(key=lambda x: x[1])

print(f"Found {len(mult_loop_files)} files:")
for fn, uf in mult_loop_files:
    print(f"  {fn} -> unroll {uf}")

Found 4 files:
  mult_loop_csynth_uf1.xml -> unroll 1
  mult_loop_csynth_uf2.xml -> unroll 2
  mult_loop_csynth_uf4.xml -> unroll 4
  mult_loop_csynth_uf8.xml -> unroll 8


## Parsing the report files

Each report file is an XML document containing detailed synthesis results. XML is a widely used format for storing data in a hierarchical tree structure, making it easy to navigate and extract specific information.

In this step, we focus on extracting two key sections:
- `Resources`: Indicates the FPGA resources actually used by the design
- `AvailableResources`: Lists the total available resources on the target FPGA

We’ll use Python’s built-in XML parsing tools to read these values and analyze utilization.

In [26]:
import xml.etree.ElementTree as ET

def parse_csynth_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    area = root.find("AreaEstimates")
    used = area.find("Resources")
    available = area.find("AvailableResources")

    def extract_resources(node):
        return {
            "DSP": int(node.find("DSP").text),
            "FF": int(node.find("FF").text),
            "LUT": int(node.find("LUT").text),
            "BRAM_18K": int(node.find("BRAM_18K").text),
            "URAM": int(node.find("URAM").text)
        }

    used_resources = extract_resources(used)
    available_resources = extract_resources(available)

    return used_resources, available_resources

In [25]:
avail_unrolled = {}
used_unrolled = {}
for fn, uf in mult_loop_files:
    xml_path = os.path.join(report_dir, fn)
    used, available = parse_csynth_xml(xml_path)
    print(f"Unroll Factor {uf}: Used DSPs = {used['DSP']}, LUTs = {used['LUT']}, FFs = {used['FF']}")
    avail_unrolled[uf] = available
    used_unrolled[uf] = used

Unroll Factor 1: Used DSPs = 3, LUTs = 250, FFs = 398
Unroll Factor 2: Used DSPs = 6, LUTs = 478, FFs = 820
Unroll Factor 4: Used DSPs = 12, LUTs = 866, FFs = 1535
Unroll Factor 8: Used DSPs = 24, LUTs = 1650, FFs = 2970


## Displaying the Synthesis Results
It is useful to parse the results in a dataframe.   There are a few key takeaways:

* The synthesis relied on three types of units: DSP slices, LUTs, and Flip-flops (FF).  We will discuss details of hardware implementation later.
* There was no use of the memory elements BRAM_18k and URAM -- these are not counted as part of the multiplication loop.  Recall that we are only looking at the loop.
* The resources grows approximataely linearly with the unrolling.
* Our vector multiplication is a tiny fraction of the overall resources.  So we will be able to do a lot more in subsequent demos.

In [28]:
import pandas as pd
df_used = pd.DataFrame.from_dict(used_unrolled, orient='index')
df_used.index.name = 'Unroll Factor'
df_used.columns.name = 'Resource'

df_percent = df_used.copy()
for res in df_used.columns:
    df_percent[res] = df_used[res] / avail_unrolled[list(avail_unrolled.keys())[0]][res] * 100

print("Resource Usage (Absolute):")
print(df_used)
print("\nResource Usage (Percentage):")
print(df_percent)

Resource Usage (Absolute):
Resource       DSP    FF   LUT  BRAM_18K  URAM
Unroll Factor                                 
1                3   398   250         0     0
2                6   820   478         0     0
4               12  1535   866         0     0
8               24  2970  1650         0     0

Resource Usage (Percentage):
Resource            DSP        FF       LUT  BRAM_18K  URAM
Unroll Factor                                              
1              0.070225  0.046793  0.058785       0.0   0.0
2              0.140449  0.096407  0.112397       0.0   0.0
4              0.280899  0.180469  0.203631       0.0   0.0
8              0.561798  0.349182  0.387980       0.0   0.0


Resource,DSP,FF,LUT,BRAM_18K,URAM
Unroll Factor,,,,,
1,0.070225,0.046793,0.058785,0.0,0.0
2,0.140449,0.096407,0.112397,0.0,0.0
4,0.280899,0.180469,0.203631,0.0,0.0
8,0.561798,0.349182,0.387980,0.0,0.0
